# 네이버 뉴스 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import concurrent.futures
from datetime import datetime, timedelta
import pandas as pd

# 날짜 범위 설정
start_date = datetime.strptime("20230901", "%Y%m%d")
end_date = datetime.strptime("20240901", "%Y%m%d")

# 날짜 리스트 생성
date_list = [(start_date + timedelta(days=x)).strftime("%Y%m%d") for x in range((end_date - start_date).days + 1)]

# 장르별 section_id 리스트 생성
section_ids = ['731', '226', '227', '732', '283', '229', '228', '230']

# section_id 리스트 네임
    # 731 : 모바일 
    # 226 : 인터넷/SNS 
    # 227 : 통신/뉴미디어 
    # 732 : 보안/해킹 
    # 283 : 컴퓨터 
    # 229 : 게임/리뷰 
    # 228 : 과학 일반 
    # 230 : IT 일반

# 함수: 개별 URL에서 제목, 기자 이름 등을 가져오기
def crawl_article(url):
    result = {
        'url': url,
        'title': None,
        'author': None,
        'publication_date': None,
        'update_date': None,
        'publisher_id': 'Null',  # 기본값 설정
        'category_id': 'IT',  # 고정 값
        'platform_id': '네이버',  # 고정 값
        'content': 'Null'  # 기본값 설정
    }
    
    try:
        article_response = requests.get(url)
        article_response.raise_for_status()
        
        article_soup = BeautifulSoup(article_response.text, 'html.parser')

        # 제목 가져오기
        title_tag = article_soup.find('h2', class_='media_end_head_headline')
        result['title'] = title_tag.get_text(strip=True) if title_tag else 'Null'
        
        # 기자 이름 가져오기 (우선 <em> 태그에서 가져오기)
        author_tag = article_soup.find('em', class_='media_end_head_journalist_name')
        if author_tag:
            result['author'] = author_tag.get_text(strip=True)
        else:
            # <div class="byline"> 태그에서 가져오기
            byline_tag = article_soup.find('div', class_='byline')
            if byline_tag:
                byline_text = byline_tag.get_text(strip=True)
                if '기자' in byline_text:
                    result['author'] = byline_text.split(' 기자')[0]
                else:
                    result['author'] = 'Null'
            else:
                result['author'] = 'Null'
        
        # publication_date 가져오기
        pub_date_tag = article_soup.find('div', class_='media_end_head_info_datestamp_bunch')
        result['publication_date'] = pub_date_tag.find('span').get_text(strip=True) if pub_date_tag else 'Null'

        # update_date 가져오기
        datestamp_bunch = article_soup.find_all('div', class_='media_end_head_info_datestamp_bunch')
        if len(datestamp_bunch) > 1:
            result['update_date'] = datestamp_bunch[1].find('span').get_text(strip=True)
        else:
            result['update_date'] = 'Null'
        
        # publisher_id 가져오기
        publisher_tag = article_soup.find('div', class_='media_end_head_top')
        if publisher_tag:
            img_tag = publisher_tag.find('img', class_='media_end_head_top_logo_img')
            result['publisher_id'] = img_tag['alt'] if img_tag and 'alt' in img_tag.attrs else 'Null'
        else:
            result['publisher_id'] = 'Null'

        # content 가져오기
        content_tag = article_soup.find('article', id='dic_area')
        result['content'] = content_tag.get_text(strip=True) if content_tag else 'Null'

    except Exception as e:
        print(f"Failed to crawl {url}: {e}")
    
    return result

# 멀티스레딩을 이용해 각 URL에서 데이터를 크롤링
def process_date(date):
    all_results = []
    for section_id in section_ids:
        base_url = f"https://news.naver.com/breakingnews/section/105/{section_id}?date={date}"
        response = requests.get(base_url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')

        # URL 크롤링: 모든 <a href> 태그의 링크를 가져오기
        urls = []
        articles = soup.find_all('div', class_='sa_text')
        for article in articles:
            link_tag = article.find('a', class_='sa_text_title')
            if link_tag and link_tag.has_attr('href'):
                urls.append(link_tag['href'])

        # 각 URL에 대해 크롤링 수행
        with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
            # tqdm 사용하여 크롤링 진행 상황 표시
            for result in tqdm(executor.map(crawl_article, urls), desc=f'Processing articles on {date}, section {section_id}', total=len(urls), unit='article', dynamic_ncols=True, ascii=True):
                all_results.append(result)
        
        time.sleep(0.3)  # 각 section 별로 딜레이 추가

    return all_results

# 날짜별로 크롤링 진행 및 CSV 파일로 저장
all_results = []
for date in tqdm(date_list, desc='Processing dates', unit='date', dynamic_ncols=True, ascii=True):
    daily_results = process_date(date)
    all_results.extend(daily_results)

# DataFrame으로 변환하여 CSV 파일로 저장
df = pd.DataFrame(all_results)
df.to_csv('naver_news_crawled_data.csv', index=False, encoding='utf-8-sig')

print("크롤링이 완료되었고, 데이터가 'naver_news_crawled_data.csv' 파일에 저장되었습니다.")
